In [5]:
import matplotlib as plt
from scipy.linalg import solve_banded
import numpy as np


import os


This simulation solves the bounce-averaged pure pitch-angle diffusion equation using the Crank-Nicolson method. All computations are done in Gaussian units.

In [6]:
# general sim parameters for t and a, important constants

N_alpha = 200
a = np.linspace(0, np.pi/2, num=N_alpha)
da = a[1]-a[0]

dt = 0.1
stop_time = 60      # should be a multiple of 5
t_range = np.linspace(0, stop_time, num=int(stop_time/dt))


# constants
mec2 = 511000           # keV
me = 9.109E-28          # g
c = 29979245800         # cm/s
Bo = 0.312              # G
Re = 6371 * 1000 * 100  # cm

Talpha = 1.3 - 0.56*np.sin(a)
sina = np.sin(a)
sin2a = np.sin(2*a)
cosa = np.cos(a)

dirac90 = np.zeros(a.shape)
dirac90[-1] = 1


In [7]:
# particle conditions/characteristics

# B-field related
L = 5   
a_L = np.asin(np.sqrt(1/L**3 * 1/np.sqrt(4-3/L)))   # radians

# particle energy related
E_k = 50000                 # keV
gam_rel = 1 + E_k/mec2
v = c * np.sqrt(1 - 1/(gam_rel)**2)     # cm/s

# characteristic loss timescale
t_L = Re * L/v * Talpha     # s

In [8]:
# diffusion and simulation conditions

# diffusion coefficient as a function of a
Daa = 0.1 / np.cos(a * np.heaviside(a-a_L, np.zeros(a.shape)))

# loss coefficient as a function of a
L = 1/t_L * np.heaviside(-(a-a_L), np.ones(a.shape))

# source term as a function of a
S = 1 * dirac90

# distribution initial condition
f0 = 0.1 * np.ones(a.shape)     # uniform distribution; counts/rad
N0 = sum(f0)                     # number of particles initially; if S=L=0, this should be conserved

f = f0

In [ ]:
# solver outputs

save_frequency = 5      # save f for plotting every 5s of simulation time
save_times = np.linspace(0, stop_time, num=int(stop_time/save_frequency))
saved_f = np.zeros((len(a), len(save_times)))
saved_f
steady_state = f    

N = np.zeros(t_range.shape)     # number of particles; saved at all simulation times

# output directory
RUN_NAME = "50keV_L5"
PLOTS_DIRECTORY = "plots/" + RUN_NAME + "/"
DATA_DIRECTORY = "data/" + RUN_NAME + "/"

if not os.path.exists(PLOTS_DIRECTORY) or not os.path.isdir(PLOTS_DIRECTORY):
    os.mkdirs(PLOTS_DIRECTORY)

if not os.path.exists(DATA_DIRECTORY) or not os.path.isdir(DATA_DIRECTORY):
    os.mkdirs(DATA_DIRECTORY)

FileNotFoundError: [Errno 2] No such file or directory: 'data/50keV_L5/'

In [ ]:
# set up and solve fat matrix inversions for each timestep

save_col = 0
for time, i in zip(t_range, range(len(t_range))):
    if time in save_times:
        saved_f[:,save_col] = f     # save f @ t_n
        save_col = save_col + 1
    N[i] = sum(f)

    # compute coefficients for each alpha

    # solve_banded and get f @ t_n+1

    print(time)